#  Boiler efficiency

In [ ]:
from gekko import GEKKO

%load_ext autoreload

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

from rhc_analysis import BoilerEfficiency, Learner, Model, Comfort, Simulator

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline
%matplotlib widget

from scipy.interpolate import RectBivariateSpline, bisplrep

from nfh_utils import *

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Initialize the class with the path to your Parquet file

boiler_returntemp_load_efficiency_file_path = 'boiler_returntemp_load_efficiency.parquet'
boiler_efficiency = BoilerEfficiency(boiler_returntemp_load_efficiency_file_path)

In [ ]:
boiler_models = [
    'Remeha Avanta Ace 28c',
    'Remeha Avanta Ace 35c',
    'Remeha Calenta Ace 28c',
    'Remeha Calenta Ace 40c',
    'Remeha Calenta Ace 40L',
    'Remeha Tzerra Ace 28c',
    'Remeha Tzerra Ace 39c',
    'Remeha Tzerra Ace Matic 35c'
]

In [ ]:
# brand_model = 'Remeha Calenta Ace 40L'
brand_model = 'Remeha Tzerra Ace Matic 35c'


In [ ]:
try:
    df_boiler_efficiency = pd.read_parquet(
        boiler_returntemp_load_efficiency_file_path,
        engine='pyarrow',
        dtype_backend='numpy_nullable'
    )
except Exception as e:
    raise IOError(f"Error reading Parquet file: {e}")

In [ ]:
df_boiler_efficiency.info()

In [ ]:
boiler_specific_efficiency = df_boiler_efficiency.loc[brand_model]

data_gas_load__pct = np.asarray(boiler_specific_efficiency.index.get_level_values('rounded_load__pct').unique().astype(float))
data_temp_ret__degC = np.asarray(boiler_specific_efficiency.index.get_level_values('rounded_temp_ret__degC').unique().astype(float))
data_eta_ch_hhv__W0 = np.asarray(boiler_specific_efficiency.unstack(level='rounded_temp_ret__degC').values.astype(float))


## Plot curve from Remeha

In [ ]:
for brand_model in boiler_models:
    boiler_specific_efficiency = df_boiler_efficiency.loc[brand_model]
    
    data_gas_load__pct = np.asarray(boiler_specific_efficiency.index.get_level_values('rounded_load__pct').unique().astype(float))
    data_temp_ret__degC = np.asarray(boiler_specific_efficiency.index.get_level_values('rounded_temp_ret__degC').unique().astype(float))
    data_eta_ch_hhv__W0 = np.asarray(boiler_specific_efficiency.unstack(level='rounded_temp_ret__degC').values.astype(float))
    
    # Generate meshgrid for plotting
    temp_ret_vals, gas_load_vals = np.meshgrid(data_temp_ret__degC, data_gas_load__pct)
    
    remeha_data_eta_ch_hhv__W0 = data_eta_ch_hhv__W0
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(temp_ret_vals, gas_load_vals, remeha_data_eta_ch_hhv__W0 , cmap='turbo')
    ax.set_xlabel('Return Temperature [°C]')
    ax.set_ylabel('Gas Load [%]')
    ax.set_zlabel('Efficiency (hhv) [W⁰]')
    plt.title(f"{brand_model} curve", fontsize=16)
    plt.show()

## use scipy.interpolate.RectBivariateSpline

In [ ]:
gas_load_eval__pct = 65
temp_ret_range__degC = np.linspace(20,70,500)
gas_load_range__pct = np.full_like(temp_ret_range__degC, gas_load_eval__pct)

In [ ]:
eta_scipy = boiler_efficiency.get_efficiency_hhv_interpolator(brand_model)

In [ ]:
eta_scipy(40,50)

In [ ]:
eta_scipy_eval = [eta_scipy(gas_load_eval__pct, temp_ret__degC) for temp_ret__degC in temp_ret_range__degC]

In [ ]:
len(eta_scipy_eval)

In [ ]:
temp_ret_eval__degC = 50

In [ ]:
eta_scipy(gas_load_eval__pct, temp_ret_eval__degC)

## Can we create a GEKKO bspline?

In [ ]:
%%time 
# Evaluate GEKKO spline
from gekko import GEKKO
m = GEKKO(remote=False)

gas_load__pct = m.MV(value=gas_load_range__pct)
gas_load__pct.STATUS = 0  # No optimization
gas_load__pct.FSTATUS = 1 # Use the measured values
    
temp_ret__degC = m.MV(value=temp_ret_range__degC)
temp_ret__degC.STATUS = 0  # No optimization
temp_ret__degC.FSTATUS = 1 # Use the measured values

eta_ch_hhv__W0 = m.Var()
kx=3
ky=3
m.bspline(gas_load__pct,temp_ret__degC, eta_ch_hhv__W0, 
          data_gas_load__pct, data_temp_ret__degC, data_eta_ch_hhv__W0, 
          data=True,
          kx=kx,
          ky=ky,
          # sf=0.1
          sf=None
         )
m.Obj(eta_ch_hhv__W0)
m.options.IMODE=2
m.solve(disp=False)

##  Plot GEKKO bspline() based Remeha data

In [ ]:
%%time 
for brand_model in boiler_models:
    boiler_specific_efficiency = df_boiler_efficiency.loc[brand_model]
    
    data_gas_load__pct = np.asarray(boiler_specific_efficiency.index.get_level_values('rounded_load__pct').unique().astype(float))
    data_temp_ret__degC = np.asarray(boiler_specific_efficiency.index.get_level_values('rounded_temp_ret__degC').unique().astype(float))
    data_eta_ch_hhv__W0 = np.asarray(boiler_specific_efficiency.unstack(level='rounded_temp_ret__degC').values.astype(float))
    
    # Evaluate GEKKO spline
    from gekko import GEKKO
    m = GEKKO(remote=False)
    
    # # Create grid for gas load and return temperature
    # grid_gas_load__pct, grid_temp_ret__degC = np.meshgrid(data_gas_load__pct, data_temp_ret__degC, indexing="ij")
    
    # Define the interpolation factor (e.g., 10 times more points); set to 1 to get MAE and RMSE value
    interp_factor = 1
    
    # Generate finer grids
    gas_load_vals = np.linspace(min(data_gas_load__pct), max(data_gas_load__pct), len(data_gas_load__pct) * interp_factor)
    temp_ret_vals = np.linspace(min(data_temp_ret__degC), max(data_temp_ret__degC), len(data_temp_ret__degC) * interp_factor)
    
    # Create a meshgrid with the new, finer resolution
    grid_gas_load__pct, grid_temp_ret__degC = np.meshgrid(gas_load_vals, temp_ret_vals, indexing="ij") 
    
    # Define Manipulated Variables (MVs)
    # Gas load percentage
    gas_load__pct = m.MV(value=grid_gas_load__pct.flatten())
    gas_load__pct.STATUS = 0  # No optimization
    gas_load__pct.FSTATUS = 1 # Use the measured values
    
    # Return temperature in °C
    temp_ret__degC = m.MV(value=grid_temp_ret__degC.flatten())  
    temp_ret__degC.STATUS = 0  # No optimization
    temp_ret__degC.FSTATUS = 1 # Use the measured value
    
    eta_ch_hhv__W0 = m.Var()
    kx=3
    ky=3
    m.bspline(gas_load__pct, temp_ret__degC, eta_ch_hhv__W0, 
              data_gas_load__pct, data_temp_ret__degC, data_eta_ch_hhv__W0, 
              data=True,
              kx=kx,
              ky=ky,
              # sf=0.1
              sf=None
             )
    m.Obj(eta_ch_hhv__W0)
    m.options.IMODE=2
    m.solve(disp=False)
    
    grid_eta_ch_hhv__W0 = np.asarray(eta_ch_hhv__W0.value).astype(float).reshape(len(gas_load_vals), len(temp_ret_vals))
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(grid_gas_load__pct, grid_temp_ret__degC, grid_eta_ch_hhv__W0, cmap='turbo')
    ax.set_xlabel('Gas Load [%]')
    ax.set_ylabel('Return Temperature [°C]')
    ax.set_zlabel('Efficiency (hhv) [W⁰]')
    
    # Flip the Return Temperature axis so lower values are on the left
    ax.invert_xaxis()
    ax.view_init(elev=30, azim=30)  # Rotate for better visibility
    fig.suptitle(f"{brand_model} GEKKO bspline curve", fontsize=16)
    if grid_eta_ch_hhv__W0.flatten().shape == data_eta_ch_hhv__W0.flatten().shape:
        ax.set_title(f"MAE = {mae(grid_eta_ch_hhv__W0.flatten(), data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]; RMSE = {rmse(grid_eta_ch_hhv__W0.flatten(), data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]")
    plt.show()

    m.cleanup()

In [ ]:
if grid_eta_ch_hhv__W0.flatten().shape == data_eta_ch_hhv__W0.flatten().shape:
    print(f"MAE = {mae(grid_eta_ch_hhv__W0.flatten(), data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]")
    print(f"RMSE = {rmse(grid_eta_ch_hhv__W0.flatten(), data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]")

##  Can we create a bspline based on RectBivariateSpline knots and coeffs?

In [ ]:
# Create interpolator
interpolator = RectBivariateSpline(
    data_gas_load__pct,
    data_temp_ret__degC,
    data_eta_ch_hhv__W0, 
    bbox=[
          data_gas_load__pct.min(), data_gas_load__pct.max(),
          data_temp_ret__degC.min(), data_temp_ret__degC.max(),
         ]
)

# Get knots and coefficients
x_knots, y_knots = interpolator.get_knots()
coeffs= interpolator.get_coeffs()
coeffs_2d = coeffs.reshape((x_knots.shape[0] - kx - 1, y_knots.shape[0] - ky - 1))

In [ ]:
%%time 
# Evaluate GEKKO spline based on RectBivariateSpline knots and coefficients
from gekko import GEKKO
m2 = GEKKO(remote=False)

gas_load__pct = m2.MV(value=gas_load_range__pct, name='gas_load__pct')
gas_load__pct.STATUS = 0  # No optimization
gas_load__pct.FSTATUS = 1 # Use the measured values
    
temp_ret__degC = m2.MV(value=temp_ret_range__degC, name='temp_ret__degC')
temp_ret__degC.STATUS = 0  # No optimization
temp_ret__degC.FSTATUS = 1 # Use the measured values

rbs_eta_hhv__W0 = m2.Var(name='rbs_eta_hhv__W0')
kx=3
ky=3
m2.bspline(gas_load__pct, temp_ret__degC, rbs_eta_hhv__W0, 
          x_knots, y_knots, coeffs, 
          data=False,
          kx=kx,
          ky=ky
         )
m2.Obj(rbs_eta_hhv__W0)
m2.options.IMODE=2
m2.solve(disp=False)

##  BSpline based on bisplrep knots and coeffs

In [ ]:
# Check for NaNs in each array and print detailed results
has_nan_x = np.any(np.isnan(data_gas_load__pct))
has_nan_y = np.any(np.isnan(data_temp_ret__degC))
has_nan_z = np.any(np.isnan(data_eta_ch_hhv__W0))

print(f"NaNs in data_gas_load__pct: {has_nan_x}")
print(f"NaNs in data_temp_ret__degC: {has_nan_y}")
print(f"NaNs in data_eta_ch_hhv__W0: {has_nan_z}")

# Optionally display the indices where NaNs are found
if has_nan_x:
    print("Indices with NaNs in data_gas_load__pct:", np.where(np.isnan(data_gas_load__pct)))
if has_nan_y:
    print("Indices with NaNs in data_temp_ret__degC:", np.where(np.isnan(data_temp_ret__degC)))
if has_nan_z:
    print("Indices with NaNs in data_eta_ch_hhv__W0:", np.where(np.isnan(data_eta_ch_hhv__W0)))


In [ ]:
# Meshgrid and flattening
x, y = np.meshgrid(data_gas_load__pct, data_temp_ret__degC, indexing='ij')
x_flat = x.ravel()
y_flat = y.ravel()
z_flat = data_eta_ch_hhv__W0.ravel()

# Fit the bivariate spline using bisplrep
kx, ky = 3, 3  # Cubic spline degrees
smoothing_factor = 0.1  # You can adjust this value as needed

tck = bisplrep(x_flat, y_flat, z_flat, kx=kx, ky=ky, s=smoothing_factor)

# Extract knots and coefficients
x_knots, y_knots, coeffs = tck[0], tck[1], tck[2]
print(f"x_knots: {x_knots}, y_knots: {y_knots}, coeffs: {coeffs}")

In [ ]:
%%time 
# Evaluate GEKKO spline based in bisplrep() knots and coefficients
m3 = GEKKO(remote=False)

gas_load__pct = m3.MV(value=gas_load_range__pct, name='gas_load__pct')
gas_load__pct.STATUS = 0  # No optimization
gas_load__pct.FSTATUS = 1 # Use the measured values
    
temp_ret__degC = m3.MV(value=temp_ret_range__degC, name='temp_ret__degC')
temp_ret__degC.STATUS = 0  # No optimization
temp_ret__degC.FSTATUS = 1 # Use the measured values

bisplrep_eta_hhv__W0 = m3.Var(name='bisplrep_eta_hhv__W0')

m3.bspline(gas_load__pct, temp_ret__degC, bisplrep_eta_hhv__W0, 
          x_knots, y_knots, coeffs, 
          data=False,
          kx=kx,
          ky=ky
         )
m3.Obj(bisplrep_eta_hhv__W0)
m3.options.IMODE=2
m3.solve(disp=False)

##  Compare in plot

In [ ]:
remeha_data_eta_ch_hhv__w0 = np.asarray(boiler_specific_efficiency.unstack(level='rounded_temp_ret__degC').loc[gas_load_eval__pct]).astype(float)

In [ ]:
# Create a plot
plt.figure(figsize=(10, 6))

# Plot both datasets
plt.plot(temp_ret_range__degC, eta_scipy_eval, label="RectBivariateSpline (Scipy)", marker=".", linestyle="--", color="green", alpha=0.5)
# plt.plot(temp_ret__degC.value, eta_ch_hhv__W0.value, label="GEKKO bspline", marker=".", linestyle="-", color="orange", alpha=0.5)
plt.plot(temp_ret__degC.value, rbs_eta_hhv__W0.value, label="GEKKO RBS bspline", marker=".", linestyle="-", color="blue", alpha=0.5)
plt.plot(temp_ret__degC.value, bisplrep_eta_hhv__W0.value, label="GEKKO bisplrep bspline", marker="x", linestyle="-", color="red", alpha=0.5)
plt.plot(data_temp_ret__degC, remeha_data_eta_ch_hhv__w0, label="Remeha data", marker="o", linestyle="-", color="black", alpha=0.5)
# Add titles and labels
plt.title(f"Comparison of Efficiency {brand_model} at gas load {gas_load_eval__pct} %", fontsize=16)
plt.xlabel("Return temperature [°C]", fontsize=14)
plt.ylabel("Efficiency (hhv) [W⁰]", fontsize=14)

# Add grid and legend
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend(fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

## Can we estimate a piecewise, kinked curve?

In [ ]:
%%time
m = GEKKO(remote=False)

# Create grid for gas load and return temperature
grid_gas_load__pct, grid_temp_ret__degC = np.meshgrid(data_gas_load__pct, data_temp_ret__degC, indexing="ij")

# Define Manipulated Variables (MVs)
# Gas load percentage
gas_load__pct = m.MV(value=grid_gas_load__pct.flatten())
gas_load__pct.STATUS = 0  # No optimization
gas_load__pct.FSTATUS = 1 # Use the measured values

# Return temperature in °C
temp_ret__degC = m.MV(value=grid_temp_ret__degC.flatten())  
temp_ret__degC.STATUS = 0  # No optimization
temp_ret__degC.FSTATUS = 1 # Use the measured value

# Define Free Variables (FVs) for parameters to be learned
c0 = m.FV(value=55)  # Intercept for condensation temperature
c1 = m.FV(value=0.1)  # Slope for condensation temperature
c2 = m.FV(value=0.01)  
t_cond0 = m.FV(value=1)
t_cond1 = m.FV(value=0.1)
t_cond2 = m.FV(value=0.01)
t_nocond0 = m.FV(value=0.9)
t_nocond1 = m.FV(value=0.02)
g_cond0 = m.FV(value=0.0)
g_cond1 = m.FV(value=0.01)
g_cond2 = m.FV(value=0.001)
g_nocond0 = m.FV(value=0.0)
g_nocond1 = m.FV(value=0.01)
g_nocond2 = m.FV(value=0.001)
# Set options for estimation
for fv in [c0, c1, c2,
           t_cond0, t_cond1, t_cond2, 
           t_nocond0, t_nocond1, 
           g_cond0, g_cond1, g_cond2, 
           g_nocond0, g_nocond1, g_nocond2,]:
    fv.STATUS = 1  # Allow optimization
    fv.FSTATUS = 1 # Use the initial value as a hint for the solver

# Define Intermediate Variables
temp_cond__degC = m.Intermediate(c0 + c1 * gas_load__pct + c2 * gas_load__pct**2)

# efficiency below condensation temperature:  2nd degree polynomial for estimate
eta_cond_temp_ch_hhv__W0 = m.Intermediate(t_cond0 + t_cond1 * temp_ret__degC + t_cond2 * temp_ret__degC ** 2)

# efficiency above condensation temperature: no condentation: linear estimate
eta_nocond_temp_ch_hhv__W0 = m.Intermediate(t_nocond0 + t_nocond1 * temp_ret__degC)

# Efficiency estimate based on temperature
eta_temp_ch_hhv__W0 = m.Intermediate(
    m.if2(temp_ret__degC - temp_cond__degC,  # Condition: below condensation temp
          eta_cond_temp_ch_hhv__W0,  # temp_ret__degC < temp_cond__degC;
          eta_nocond_temp_ch_hhv__W0)  # temp_ret__degC >= temp_cond__degC: no consendation
)

# Ensure continuity at condensation temperature
m.Equation(
    (t_cond0 + t_cond1 * temp_cond__degC + t_cond2 * temp_cond__degC ** 2 + g_cond0 + g_cond1 * gas_load__pct + g_cond2 * gas_load__pct ** 2) ==
    (t_nocond0 + t_nocond1 * temp_cond__degC + g_nocond0 + g_nocond1 * gas_load__pct + g_nocond2 * gas_load__pct ** 2)
)

# gas-load influence on efficiency below condensation temperature (condensation)
eta_cond_gas_load_ch_hhv__W0 = m.Intermediate(eta_temp_ch_hhv__W0 + g_cond0 + g_cond1 * gas_load__pct + g_cond2 * gas_load__pct ** 2)

# gas-load influence on efficiency above condensation temperature (no condensation)
eta_nocond_gas_load_ch_hhv__W0 = m.Intermediate(eta_temp_ch_hhv__W0 + g_nocond0 + g_nocond1 * gas_load__pct + g_nocond2 * gas_load__pct ** 2)

# Efficiency estimate including gas load, based on temperature
eta_gas_load_temp_ch_hhv__W0 = m.Intermediate(
    m.if2(temp_ret__degC - temp_cond__degC,  # Condition: below condensation temp
          eta_cond_gas_load_ch_hhv__W0,  # temp_ret__degC < temp_cond__degC;
          eta_nocond_gas_load_ch_hhv__W0)  # temp_ret__degC >= temp_cond__degC: no consendation
)

# Ensure continuity at condensation temperature
m.Equation(
    (t_cond0 + t_cond1 * temp_cond__degC + t_cond2 * temp_cond__degC ** 2 + g_cond0 + g_cond1 * gas_load__pct + g_cond2 * gas_load__pct ** 2) ==
    (t_nocond0 + t_nocond1 * temp_cond__degC + g_nocond0 + g_nocond1 * gas_load__pct + g_nocond2 * gas_load__pct ** 2)
)

# Full efficiency estimate including gas load dependency
eta_ch_hhv__W0 = m.CV(value=data_eta_ch_hhv__W0.flatten())
eta_ch_hhv__W0.STATUS = 1  # Include this variable in the optimization (enabled for fitting)
eta_ch_hhv__W0.FSTATUS = 1  # Use the measured values

m.Equation(eta_ch_hhv__W0 == eta_gas_load_temp_ch_hhv__W0)


# Solve model
m.options.IMODE = 2  # Parameter estimation mode
m.options.EV_TYPE = 2      # RMSE
m.solve(disp=True)

## Plot fitted kinked curve learned from Remeha data

In [ ]:
# Generate meshgrid for plotting
temp_ret_vals, gas_load_vals = np.meshgrid(data_temp_ret__degC, data_gas_load__pct)

# Compute condensation temperature
temp_cond_vals = c0.value[0] + c1.value[0] * gas_load_vals + c2.value[0] * gas_load_vals**2

# Apply piecewise calculation
fitted_pw_poly_eta_ch_hhv__W0 = np.where(
    temp_ret_vals < temp_cond_vals,  # Below condensation temperature
    (t_cond0.value[0] + t_cond1.value[0] * temp_ret_vals + t_cond2.value[0] * temp_ret_vals**2),
    (t_nocond0.value[0] + t_nocond1.value[0] * temp_ret_vals)
)

# Add gas load dependency
fitted_pw_poly_eta_ch_hhv__W0 = np.where(
    temp_ret_vals < temp_cond_vals,  # Below condensation temperature
    fitted_pw_poly_eta_ch_hhv__W0  + (g_cond0.value[0] + g_cond1.value[0] * gas_load_vals + g_cond2.value[0] * gas_load_vals**2),
    fitted_pw_poly_eta_ch_hhv__W0  + (g_nocond0.value[0] + g_nocond1.value[0] * gas_load_vals + g_nocond2.value[0] * gas_load_vals**2),
)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(temp_ret_vals, gas_load_vals, fitted_pw_poly_eta_ch_hhv__W0 , cmap='turbo')
ax.set_xlabel('Return Temperature [°C]')
ax.set_ylabel('Gas Load [%]')
ax.set_zlabel('Efficiency (hhv) [W⁰]')
fig.suptitle(f"{brand_model} poly-fitted curve", fontsize=16)
ax.set_title(f"MAE = {mae(fitted_pw_poly_eta_ch_hhv__W0 .flatten(), data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]; RMSE = {rmse(fitted_pw_poly_eta_ch_hhv__W0 .flatten(), data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]")
plt.show()

In [ ]:
fitted_pw_poly_eta_ch_hhv__W0 .shape

In [ ]:
fitted_pw_poly_eta_ch_hhv__W0 .flatten().shape

In [ ]:
print(f"MAE = {mae(fitted_pw_poly_eta_ch_hhv__W0 .flatten(), data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]")
print(f"RMSE = {rmse(fitted_pw_poly_eta_ch_hhv__W0 .flatten(), data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]")

In [ ]:
# Print learned parameters
print(f"{brand_model} fitted condensation-kinked curve")
print(f"c0: {c0.VALUE[0]}, c1: {c1.VALUE[0]}, c2: {c2.VALUE[0]}")
print(f"t_cond0: {t_cond0.VALUE[0]}, t_cond1: {t_cond1.VALUE[0]}, t_cond2: {t_cond2.VALUE[0]}")
print(f"t_nocond0: {t_nocond0.VALUE[0]}, t_nocond1: {t_nocond1.VALUE[0]}")
print(f"g_cond0: {g_cond0.VALUE[0]}, g_cond1: {g_cond1.VALUE[0]}, g_cond2: {g_cond2.VALUE[0]}")
print(f"g_nocond0: {g_nocond0.VALUE[0]}, g_nocond1: {g_nocond1.VALUE[0]}, g_nocond2: {g_nocond2.VALUE[0]}")

print(f"MAE = {mae(eta_ch_hhv__W0, data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]")
print(f"RMSE = {rmse(eta_ch_hhv__W0, data_eta_ch_hhv__W0.flatten())*100:.2f} [%-point]")

## Alternate rouce, using scipy.optimize.curve_fit

In [ ]:
import scipy.optimize as opt

In [ ]:
# Create grid for gas load and return temperature
grid_gas_load__pct, grid_temp_ret__degC = np.meshgrid(data_gas_load__pct, data_temp_ret__degC, indexing="ij")

# Flatten the grid data for curve fitting
gas_load_flat = grid_gas_load__pct.flatten()
temp_ret_flat = grid_temp_ret__degC.flatten()
eta_hhv_flat = data_eta_ch_hhv__W0.flatten()

# Define piecewise function
def efficiency_model(X, c0, c1, t_cond0, t_cond1, t_cond2, t_nocond0, t_nocond1, g0, g1, g2):
    gas_load, temp_ret = X
    temp_cond = c0 + c1 * gas_load
    
    cond_phase = (temp_ret < temp_cond)  # Boolean mask for condensation phase

    eta_temp = np.where(
        cond_phase,  
        t_cond0 + t_cond1 * temp_ret + t_cond2 * temp_ret**2,  # Quadratic model
        t_nocond0 + t_nocond1 * temp_ret  # Linear model
    )

    return eta_temp + g0 + g1 * gas_load + g2 * gas_load**2

# Fit the model to the data
popt, _ = opt.curve_fit(efficiency_model, (gas_load_flat, temp_ret_flat), eta_hhv_flat, 
                        p0=[55, 0.1, 1, 0.1, 0.01, 0.9, 0.02, 0.0, 0.01, 0.001])

# Extract fitted coefficients
c0_fit, c1_fit, t_cond0_fit, t_cond1_fit, t_cond2_fit, t_nocond0_fit, t_nocond1_fit, g0_fit, g1_fit, g2_fit = popt

In [ ]:
print("Fitted parameters:")
print(f"c0: {c0_fit}, c1: {c1_fit}")
print(f"t_cond0: {t_cond0_fit}, t_cond1: {t_cond1_fit}, t_cond2: {t_cond2_fit}")
print(f"t_nocond0: {t_nocond0_fit}, t_nocond1: {t_nocond1_fit}")
print(f"g0: {g0_fit}, g1: {g1_fit}, g2: {g2_fit}")